# 載入相關套件

In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np 
from keras.models import Sequential
from keras.layers import Input, Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 載入資料集

In [ ]:
X_train =  np.loadtxt('/PCA/PCA_DATA/x_train_90.txt')
X_test =  np.loadtxt('/PCA/PCA_DATA/x_test_90.txt')
Y_train = np.loadtxt('/PCA/PCA_DATA/y_train_90.txt')
Y_test  =  np.loadtxt('/PCA/PCA_DATA/y_test_90.txt')

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(220717, 26)
(73598, 26)
(220717,)
(73598,)


In [ ]:
feature_num = X_train.shape[1]
print(feature_num)

76


# 資料預處理(合併資料集)

In [ ]:
X = np.concatenate((X_train, X_test), axis = 0)
Y = np.concatenate((Y_train, Y_test), axis = 0)
print(X.shape)
print(Y.shape)

(294315, 76)
(294315,)


In [ ]:
print('the shape of Y',Y.shape)
Y = np.reshape(Y, (Y.size,1))
print('the reshape of Y', Y.shape)

the shape of Y (294315,)
the reshape of Y (294315, 1)


# 分割訓練和測試資料集


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
print('訓練資料集 X:',X_train.shape)
print('訓練資料集 Y:',Y_train.shape)
print('測試資料集 X:',X_test.shape)
print('測試資料集 Y:',Y_test.shape)

訓練資料集 X: (206020, 76)
訓練資料集 Y: (206020, 1)
測試資料集 X: (88295, 76)
測試資料集 Y: (88295, 1)


# 模型建立

In [ ]:
model = Sequential()
InputLayer = Input( batch_input_shape=(None, feature_num ), name="dense_input", dtype=tf.float32, sparse=False, ragged=False)
model.add(InputLayer)
model.add(Dense(512, batch_input_shape=(None, feature_num ), name="dense", dtype=tf.float32, activation="relu"))
model.add(Dense(256, name="dense_1", dtype=tf.float32, activation="relu"))
model.add(Dense(128, name="dense_2", dtype=tf.float32, activation="relu"))
model.add(Dense(39, name="dense_3", dtype=tf.float32, activation="softmax"))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy' , metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               39424     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 39)                5031      
                                                                 
Total params: 208,679
Trainable params: 208,679
Non-trainable params: 0
_________________________________________________________________


# 模型訓練

In [ ]:
history = model.fit(X_train, Y_train, epochs=1000, batch_size=512 , validation_split=0.05)

Epoch 1/1000
383/383 [==============================] - 8s 20ms/step - loss: 0.5094 - sparse_categorical_accuracy: 0.8789 - val_loss: 0.3743 - val_sparse_categorical_accuracy: 0.8932
Epoch 2/1000
383/383 [==============================] - 7s 18ms/step - loss: 0.3770 - sparse_categorical_accuracy: 0.8937 - val_loss: 0.3520 - val_sparse_categorical_accuracy: 0.9005
Epoch 3/1000
383/383 [==============================] - 7s 18ms/step - loss: 0.3604 - sparse_categorical_accuracy: 0.8969 - val_loss: 0.3436 - val_sparse_categorical_accuracy: 0.9022
Epoch 4/1000
383/383 [==============================] - 7s 18ms/step - loss: 0.3495 - sparse_categorical_accuracy: 0.8990 - val_loss: 0.3361 - val_sparse_categorical_accuracy: 0.9015
Epoch 5/1000
383/383 [==============================] - 7s 18ms/step - loss: 0.3430 - sparse_categorical_accuracy: 0.8997 - val_loss: 0.3305 - val_sparse_categorical_accuracy: 0.9032
Epoch 6/1000
383/383 [==============================] - 7s 18ms/step - loss: 0.3374 -

KeyboardInterrupt: ignored

# 圖形化訓練過程

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# 測試模型精準度

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
y_pred = model.predict(X_test)
y_true = Y_test

In [ ]:
print("y_true's shape: ", y_true.shape)
print("y_pred's shape: ", y_pred.shape)

In [ ]:
pred = np.argmax(y_pred, axis = 1)
pred = np.reshape(pred, (len(pred), 1))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, pred))

# 製作混淆矩陣


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import confusion_matrix
fig, ax = plt.subplots(figsize=(25, 25))
mat = confusion_matrix(y_true, pred)
sns.heatmap(mat,square= True, annot=True, cbar= True, ax=ax)
plt.xlabel("predicted value")
plt.ylabel("true value")
plt.show()